In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=0aaff60b6169c9d9cf217826d43b177b0ef4537766f8876c0c74fac2a462abbc
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.6MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F2 '''
def F2(X):
    f = bn.F2()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 6124.388726646325 ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_2_2000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_2_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=947658, Mon Apr 13 21:45:36 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 5.833379419981202e+07 1.0e+00 2.41e+00  2e+00  2e+00 0:00.0
    2     34 5.047271790364108e+07 1.0e+00 2.34e+00  2e+00  2e+00 0:00.1
    3     51 3.737537059143401e+07 1.0e+00 2.29e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [91120997.0038011] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [55132463.74437813] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   87   1479 -4.391079886407989e+07 1.3e+00 1.21e+00  1e+00  1e+00 0:03.1
  100   1700 -6.135087283228213e+07 1.3e+00 9.78e-01  9e-01  1e+00 0:03.6
  200   3400 -1.272542192646199e+08 1.3e+00 1.70e-01  2e-01  2e-01 0:07.2
  300   5100 -1.294789670741583e+08 1.4e+00 3.07e-02  3e-02  3e-02 0:10.8
  400   6800 -1.295539642193957e+08 1.4e+00 6.23e-03  5e-03  6e-03 0:14.4
  500   8500 -1.295576351918226e+08 1.4e+00 1.46e-03  1e-03  1e-03 0:18.1
  600  10200 -1.295578417678344e+08 1.5e+00 3.10e-04  3e-04  3e-04 0:21.7
  700  11900 -1.295578483865584e+08 1.5e+00 6.13e-05  5e-05  6e-05 0:25.3
  800  13600 -1.295578486417678e+08 1.5e+00 1.24e-05  9e-06  1e-05 0:29.0
  900  15300 -1.295578486525839e+08 1.6e+00 2.24e-06  2e-06  2e-06 0:32.6
 1000  17000 -1.295578486529787e+08 1.6e+00 3.47e-07  3e-07  3e-07 0:36.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.295578486529880e+08 1.6e+00 7.47e-08  5e-08  7e-08 0:39.8
 1200  20400 -1.295578486529883e+08 1.6e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [92935825.15909508] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=92935825.159095, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=92935825.159095, sigma=2.41e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

    2     34 4.063226204608534e+07 1.0e+00 2.34e+00  2e+00  2e+00 0:00.1
    3     51 3.059770983796784e+07 1.0e+00 2.28e+00  2e+00  2e+00 0:00.1


...

   84   1428 -5.929048371117964e+07 1.3e+00 1.11e+00  1e+00  1e+00 0:03.1
  100   1700 -6.991542733795205e+07 1.3e+00 8.49e-01  8e-01  9e-01 0:03.7
  200   3400 -1.272326696669872e+08 1.3e+00 1.68e-01  2e-01  2e-01 0:07.5
  300   5100 -1.294494921708497e+08 1.4e+00 3.56e-02  3e-02  4e-02 0:11.3
  400   6800 -1.295536021556200e+08 1.4e+00 6.48e-03  6e-03  6e-03 0:15.1
  500   8500 -1.295576831169286e+08 1.4e+00 1.35e-03  1e-03  1e-03 0:18.9
  600  10200 -1.295578424803496e+08 1.5e+00 2.72e-04  2e-04  3e-04 0:22.6
  700  11900 -1.295578484456187e+08 1.5e+00 4.98e-05  4e-05  5e-05 0:26.4
  800  13600 -1.295578486459232e+08 1.5e+00 9.68e-06  7e-06  9e-06 0:30.2
  900  15300 -1.295578486527718e+08 1.5e+00 1.57e-06  1e-06  2e-06 0:34.0
 1000  17000 -1.295578486529808e+08 1.6e+00 3.24e-07  2e-07  3e-07 0:37.8
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.295578486529881e+08 1.6e+00 6.43e-08  5e-08  6e-08 0:41.6
 1200  20400 -1.295578486529883e+08 1.6e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [92935825.15909508] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=92935825.159095, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=92935825.159095, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iterati

    2     34 3.827853910699722e+07 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 3.088069994177227e+07 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [73623411.95804161] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   76   1292 1.207093777892404e+07 1.3e+00 1.52e+00  1e+00  2e+00 0:03.1
  100   1700 1.269566840933487e+07 1.4e+00 1.41e+00  1e+00  1e+00 0:04.1
  200   3400 4.997833492956586e+06 1.5e+00 7.80e-01  7e-01  8e-01 0:08.3
  300   5100 1.653969598355945e+06 1.6e+00 5.70e-01  5e-01  6e-01 0:12.5
  400   6800 -9.990954722313094e+05 1.7e+00 3.03e-01  2e-01  3e-01 0:16.6
  500   8500 -2.531950583417062e+06 1.8e+00 1.74e-01  1e-01  2e-01 0:20.8
  600  10200 -3.015289796225890e+06 2.0e+00 1.07e-01  8e-02  1e-01 0:24.9
  700  11900 -3.306273705747166e+06 2.5e+00 7.54e-02  5e-02  9e-02 0:29.1
  800  13600 -3.488693228011040e+06 3.0e+00 6.94e-02  5e-02  1e-01 0:33.3
  900  15300 -3.612510979837051e+06 3.3e+00 4.85e-02  3e-02  8e-02 0:37.4
 1000  17000 -3.719952252829533e+06 3.6e+00 5.95e-02  4e-02  1e-01 0:41.6
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -3.808076986763586e+06 4.1e+00 4.77e-02  3e-02  9e-02 0:45.7
 1200  20400 -3.872403182264248e+06 4.4e+00 

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F2_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F2(X_Values.iloc[i,:100])
    SVM_Fun [i] = F2(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F2(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([28051161.0390439]), count=array([1])) 0.39299812321134153
SVM
ModeResult(mode=array([53249197.79823326]), count=array([1])) 30697511.94254854
ELN
ModeResult(mode=array([1071176.5559164]), count=array([1])) 276361.85281689477
